<a href="https://colab.research.google.com/github/hariharan29200/Python_exe/blob/main/cheating_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Emotion Detection Model (fer library)**: This library provides a pre-trained model for facial expression analysis.

**Behavior Analysis**: We will track eye movements and body posture using dlib and OpenCV

In [1]:
pip install opencv-python opencv-python-headless dlib fer imutils

In [2]:
!wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
!bzip2 -d shape_predictor_68_face_landmarks.dat.bz2

--2025-03-12 02:15:10--  http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
Resolving dlib.net (dlib.net)... 107.180.26.78
Connecting to dlib.net (dlib.net)|107.180.26.78|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2 [following]
--2025-03-12 02:15:10--  https://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
Connecting to dlib.net (dlib.net)|107.180.26.78|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64040097 (61M)
Saving to: ‘shape_predictor_68_face_landmarks.dat.bz2’

shape_predictor_68_ 100%[===================>]  61.07M  37.0MB/s    in 1.7s    

2025-03-12 02:15:12 (37.0 MB/s) - ‘shape_predictor_68_face_landmarks.dat.bz2’ saved [64040097/64040097]

bzip2: Output file shape_predictor_68_face_landmarks.dat already exists.


In [3]:
!ls -l shape_predictor_68_face_landmarks.dat

-rw-r--r-- 1 root root 99693937 Jul 24  2015 shape_predictor_68_face_landmarks.dat


In [4]:
import cv2
import dlib
import numpy as np
import os
from google.colab import files

# Load face detector and facial landmark predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('/content/shape_predictor_68_face_landmarks.dat')

def analyze_video(video_path):
    cap = cv2.VideoCapture(video_path)
    frame_count = 0
    cheat_indicators = []
    emotion_scores = {"neutral": 0, "happy": 0, "sad": 0, "angry": 0, "surprised": 0, "fear": 0}
    total_frames = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = detector(gray)

        for face in faces:
            landmarks = predictor(gray, face)
            features = extract_features(landmarks)
            cheat_score = detect_cheating(features)
            detected_emotion = analyze_emotions(features)
            cheat_indicators.append(cheat_score)
            emotion_scores[detected_emotion] += 1
            total_frames += 1

    cap.release()
    avg_cheat_score = np.mean(cheat_indicators)

    # Normalize emotion scores
    for emotion in emotion_scores:
        emotion_scores[emotion] = round((emotion_scores[emotion] / total_frames) * 100, 2) if total_frames > 0 else 0

    return avg_cheat_score > 0.5, emotion_scores

def extract_features(landmarks):
    # Extract key features: eye movement, blink rate, mouth movement
    eye_aspect_ratio = (landmarks.part(37).y - landmarks.part(41).y) / (landmarks.part(36).x - landmarks.part(39).x)
    mouth_aspect_ratio = (landmarks.part(66).y - landmarks.part(62).y) / (landmarks.part(60).x - landmarks.part(64).x)
    return [eye_aspect_ratio, mouth_aspect_ratio]

def detect_cheating(features):
    eye_aspect, mouth_aspect = features
    if eye_aspect < 0.2 or mouth_aspect > 0.6:
        return 1
    return 0

def analyze_emotions(features):
    eye_aspect, mouth_aspect = features
    if mouth_aspect > 0.6:
        return "surprised"
    elif eye_aspect < 0.2:
        return "sad"
    elif mouth_aspect > 0.4:
        return "happy"
    elif eye_aspect < 0.1 and mouth_aspect < 0.2:
        return "fear"
    elif eye_aspect < 0.3 and mouth_aspect < 0.3:
        return "neutral"
    else:
        return "angry"

# Upload and analyze videos
uploaded = files.upload()
for video_file in uploaded.keys():
    is_cheating, emotion_scores = analyze_video(video_file)
    print(f'\nAnalysis for {video_file}:')
    print(f'Cheating Detected: {"Yes" if is_cheating else "No"}')
    print("Emotion Scores:")
    for emotion, score in emotion_scores.items():
        print(f'  {emotion.capitalize()}: {score}%')


Saving 1741601588042.webm to 1741601588042.webm

Analysis for 1741601588042.webm:
Cheating Detected: No
Emotion Scores:
  Neutral: 60.92%
  Happy: 0.0%
  Sad: 14.94%
  Angry: 24.14%
  Surprised: 0.0%
  Fear: 0.0%
